# Flash-Ops NL2SQL Visualization

This notebook provides visualization tools for:
1. FAISS vector embeddings (table similarity)
2. Query pipeline performance
3. Table clustering results

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
sns.set_style('whitegrid')

## 1. Load FAISS Index and Metadata

In [ ]:
from app.services import get_vector_store

# Load vector store
vector_store = get_vector_store()

print(f"Index loaded: {vector_store.loaded}")
print(f"Total tables: {len(vector_store.metadata) if vector_store.metadata else 0}")

## 2. Visualize Table Embeddings (PCA)

In [ ]:
if vector_store.loaded:
    # Get embeddings from FAISS index
    n_vectors = vector_store.index.ntotal
    embeddings = np.zeros((n_vectors, vector_store.index.d))
    
    for i in range(n_vectors):
        embeddings[i] = vector_store.index.reconstruct(i)
    
    # Apply PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)
    
    # Create DataFrame
    df_viz = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'table': [m['table_name'] for m in vector_store.metadata],
        'category': [m.get('category', 'unknown') for m in vector_store.metadata]
    })
    
    # Plot
    fig = px.scatter(
        df_viz,
        x='x',
        y='y',
        color='category',
        hover_data=['table'],
        title='Table Embeddings Visualization (PCA)',
        width=1000,
        height=700
    )
    fig.show()
else:
    print("Vector store not loaded. Generate embeddings first.")

## 3. Table Similarity Heatmap

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

if vector_store.loaded:
    # Calculate similarity for first 20 tables
    n_tables = min(20, len(vector_store.metadata))
    sample_embeddings = embeddings[:n_tables]
    
    similarity_matrix = cosine_similarity(sample_embeddings)
    
    # Plot heatmap
    table_names = [m['table_name'][:20] for m in vector_store.metadata[:n_tables]]
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        similarity_matrix,
        xticklabels=table_names,
        yticklabels=table_names,
        cmap='viridis',
        annot=False,
        cbar_kws={'label': 'Cosine Similarity'}
    )
    plt.title('Table Similarity Heatmap (Top 20 Tables)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 4. Test Query and Visualize Results

In [ ]:
import requests

# Test query
test_query = "show all active employees"

response = requests.post(
    "http://localhost:8000/api/v1/query/",
    json={"query": test_query}
)

if response.status_code == 200:
    result = response.json()
    
    print(f"Status: {result['status']}")
    print(f"Tables used: {result['tables_used']}")
    print(f"Tier: {result['tier']}")
    print(f"Confidence: {result['confidence']:.3f}")
    print(f"Execution time: {result['execution_time_ms']}ms")
    print(f"\nGenerated SQL:\n{result['sql_generated']}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 5. Clustering Visualization

In [ ]:
from app.agents import create_table_clustering

if vector_store.loaded and len(vector_store.metadata) > 5:
    # Sample tables for clustering demo
    sample_tables = vector_store.metadata[:30]
    
    # Add scores (mock)
    for i, table in enumerate(sample_tables):
        table['score'] = 0.9 - (i * 0.02)
    
    # Cluster
    clustering = create_table_clustering()
    clusters = clustering.cluster_tables(sample_tables)
    
    print(f"Created {len(clusters)} clusters:")
    for i, cluster in enumerate(clusters):
        print(f"\nCluster {i+1} ({len(cluster)} tables):")
        for table in cluster[:5]:  # Show first 5
            print(f"  - {table['table_name']} (score: {table.get('score', 0):.2f})")

## 6. Performance Metrics

In [ ]:
# Test multiple queries and measure performance
test_queries = [
    "show all employees",
    "count active users",
    "sales by department",
    "employee with highest salary"
]

results = []

for query in test_queries:
    response = requests.post(
        "http://localhost:8000/api/v1/query/",
        json={"query": query}
    )
    
    if response.status_code == 200:
        data = response.json()
        results.append({
            'query': query,
            'status': data['status'],
            'tier': data['tier'],
            'execution_time_ms': data['execution_time_ms'],
            'confidence': data['confidence']
        })

# Visualize
df_perf = pd.DataFrame(results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Execution time
axes[0].bar(range(len(df_perf)), df_perf['execution_time_ms'])
axes[0].set_xlabel('Query')
axes[0].set_ylabel('Execution Time (ms)')
axes[0].set_title('Query Execution Time')
axes[0].set_xticks(range(len(df_perf)))
axes[0].set_xticklabels(range(1, len(df_perf)+1))

# Confidence
axes[1].bar(range(len(df_perf)), df_perf['confidence'], color='orange')
axes[1].set_xlabel('Query')
axes[1].set_ylabel('Confidence Score')
axes[1].set_title('Query Confidence Scores')
axes[1].set_xticks(range(len(df_perf)))
axes[1].set_xticklabels(range(1, len(df_perf)+1))
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\nPerformance Summary:")
print(df_perf.to_string(index=False))